# Idea
Do some training

In [1]:
from pathlib import Path

import numpy as np
import pandas as pd
import pickle

In [2]:
import sys
repo_base_path = Path("./../../").resolve()
assert str(repo_base_path).endswith("csg_is"), f"{repo_base_path} is not a valid path to the CSG_IS repository" 

sys.path.append(str(repo_base_path))

# from py_dataset import get_all_files_df
# from py_dataset import read_in_files
from py_dataset import feature_plotting
# from py_dataset import feature_selection
# from py_dataset import net_feature_engineering
# from py_dataset import entropy_feature_engineering
# from py_dataset.classes import DataSource
from py_dataset import sys_func



In [4]:
df = pd.read_csv('../data/all_df.csv')
df.set_index("timestamp", inplace=True)

In [7]:
df["label"].unique()

array(['1_normal', nan, '2_ransomware', '3_thetick', '4_bashlite',
       '5_httpbackdoor', '6_beurk', '7_backdoor', '8_bdvl', '9_xmrig'],
      dtype=object)

In [1]:
with open("rfe_selected_columns.npy","r") as f:
    col_str= f.read()
col_str

"'ext4:ext4_da_update_reserve_space_FLS_data','signal:signal_deliver','net:net_dev_xmit','sched:sched_process_exec_KERN_data','jbd2:jbd2_start_commit_KERN_data','tcp:tcp_probe_KERN_data','ext4:ext4_unlink_enter_RES_data','write_merge','sched:sched_stat_runtime','pkt_out','cache-references','ext4:ext4_free_inode_RES_data','armv7_cortex_a15/br_mis_pred/','iowrite','pkt_in','memory','write_kbs','connect','getppid','mkdir','getegid32','statfs64','geteuid32','recv','setitimer','fstat64','lgetxattr','pipe2','setgroups32','getuid32','label'"

In [2]:
cols = col_str.replace("'","").split(",")
cols

['ext4:ext4_da_update_reserve_space_FLS_data',
 'signal:signal_deliver',
 'net:net_dev_xmit',
 'sched:sched_process_exec_KERN_data',
 'jbd2:jbd2_start_commit_KERN_data',
 'tcp:tcp_probe_KERN_data',
 'ext4:ext4_unlink_enter_RES_data',
 'write_merge',
 'sched:sched_stat_runtime',
 'pkt_out',
 'cache-references',
 'ext4:ext4_free_inode_RES_data',
 'armv7_cortex_a15/br_mis_pred/',
 'iowrite',
 'pkt_in',
 'memory',
 'write_kbs',
 'connect',
 'getppid',
 'mkdir',
 'getegid32',
 'statfs64',
 'geteuid32',
 'recv',
 'setitimer',
 'fstat64',
 'lgetxattr',
 'pipe2',
 'setgroups32',
 'getuid32',
 'label']

In [10]:
df = df[cols]

### Preprocessings
- remove Null values
- drop timestamp
- labelEncode
- normalize

In [11]:
print(df.shape)
old_df = df.copy()
df.dropna(axis=0, how='any', inplace=True)

print(df.shape)

(43248, 31)
(26842, 31)


/tmp/ipykernel_136981/708996582.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(axis=0, how='any', inplace=True)


In [12]:
df.reset_index(inplace=True, drop=True)

In [13]:
label_encoder = sys_func.create_label_encoder_for_labels()
df['label'] = label_encoder.transform(df['label'])

/tmp/ipykernel_136981/3852457944.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = label_encoder.transform(df['label'])


In [14]:
df = feature_plotting.test_transform_all_to_numeric_columns(df)

In [15]:
cols_to_exclude_from_scaling = ["label"]
vectors = feature_plotting.minmax_scale_features(df,remove_outliers_iqr_all_columns=False,
                                                        cols_to_exclude_from_scaling=cols_to_exclude_from_scaling)
# print(vectors.shape)
# print(" with dropped null: ", vectors.dropna(axis=0, how='any', inplace=False).shape)

In [16]:
df["label"].unique(), vectors["label"].unique()

(array([0, 1, 2, 3, 4, 5, 6, 7, 8]), array([0, 1, 2, 3, 4, 5, 6, 7, 8]))

### Setup

In [17]:
from torch.utils.data import Dataset, DataLoader
import torch

class CustomeDataset(Dataset):
    def __init__(self, some_df:pd.DataFrame):
        
        self.x = torch.tensor(some_df.drop(columns=["label"], inplace=False).values, dtype=torch.float32)
        self.labels = torch.tensor(some_df["label"].values, dtype=torch.uint8)

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        # if torch.is_tensor(idx):
        #     idx = idx.tolist()
        # print(self.x[idx], self.labels[idx])
        return self.x[idx], self.labels[idx]

In [21]:
sys.path.append(str(repo_base_path  / "training"))

In [22]:
from mlp import MalwaresModelMLP
from torch.utils.data import DataLoader
import lightning
from sklearn.model_selection import train_test_split
from lightning import loggers as pl_loggers
from lightning.pytorch.callbacks.early_stopping import EarlyStopping
from lightning.pytorch.callbacks.model_checkpoint import ModelCheckpoint

# tensorboard = pl_loggers.TensorBoardLogger('./')

# split into train and val
train, val = train_test_split(vectors, test_size=0.2)

train_dataset = CustomeDataset(train)
train_dataloader = DataLoader(train_dataset, batch_size=500, shuffle=True, num_workers=7)

val_dataset = CustomeDataset(val)
val_dataloader = DataLoader(val_dataset, batch_size=500, shuffle=False, num_workers=7)

# trainer = lightning.Trainer(fast_dev_run=100)
monitor = "Validation/Accuracy"
early_stop_callback = EarlyStopping(monitor=monitor, patience=5, verbose=True, mode="max")
checkpoint_callback = ModelCheckpoint(save_top_k=1, monitor=monitor, mode="max")

trainer = lightning.Trainer(max_epochs=150, callbacks=[early_stop_callback, checkpoint_callback], enable_checkpointing=True) #logger=None, 


model = MalwaresModelMLP(input_size = vectors.shape[1]-1, output_size = len(label_encoder.classes_))

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [23]:
trainer.fit(model=model, train_dataloaders=train_dataloader, val_dataloaders=val_dataloader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type                      | Params
------------------------------------------------------------
0 | train_metrics | MetricCollection          | 0     
1 | val_metrics   | MetricCollection          | 0     
2 | test_metrics  | MetricCollection          | 0     
3 | cm            | MulticlassConfusionMatrix | 0     
4 | criterion     | CrossEntropyLoss          | 0     
5 | l1            | Linear                    | 930   
6 | l2            | Linear                    | 930   
7 | l3            | Linear                    | 279   
------------------------------------------------------------
2.1 K     Trainable params
0         Non-trainable params
2.1 K     Total params
0.009     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Epoch end: Validation, epoch number: 0
ValidationEpoch/CM
 tensor([[ 90,   0,   0,   0,   0,   0,   0,   0,   0],
        [  6,   0,   0,   0,   0, 130,   0,   0,   0],
        [149,   0,   0,   0,   0,   3,   0,   0,   0],
        [ 97,   0,   0,   0,   0,   0,   0,   0,   0],
        [104,   0,   0,   0,   0,   0,   0,   0,   0],
        [ 97,   0,   0,   0,   0,   0,   0,   0,   0],
        [106,   0,   0,   0,   0,   1,   0,   0,   0],
        [102,   0,   0,   0,   0,   0,   0,   0,   0],
        [115,   0,   0,   0,   0,   0,   0,   0,   0]])


/home/<User>/repos/csg_is/.venv/lib/python3.10/site-packages/lightning/pytorch/loops/fit_loop.py:298: The number of training batches (43) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Training: |          | 0/? [00:00<?, ?it/s]

...

Epoch end: Validation, epoch number: 33
ValidationEpoch/CM
 tensor([[ 77557,      0,   1369,   3943,    634,      0,    735,      0,      2],
        [     9, 108164,      1,      0,      5,    194,      0,      2,      1],
        [   322,      1, 128820,    189,     93,    208,  10702,      0,      1],
        [  4236,      1,   1504,  77108,    927,      0,   3696,      0,     75],
        [   862,      0,   1288,    797,  86542,     14,    194,      0,      2],
        [   142,     78,   1179,      0,      0,  91796,     49,     45,      0],
        [  1340,      7,  23297,   7402,    268,    161,  55080,      0,      2],
        [   162,    588,     62,      0,      0,      0,      0,  86575,      0],
        [   225,     85,    298,      7,     17,      1,      2,      0, 107720]])
Epoch end: Train, epoch number: 32
TrainEpoch/CM
 tensor([[ 77557,      0,   1369,   3943,    634,      0,    735,      0,      2],
        [     9, 108164,      1,      0,      5,    194,      0,     

Validation: |          | 0/? [00:00<?, ?it/s]

Epoch end: Validation, epoch number: 34
ValidationEpoch/CM
 tensor([[ 80085,      0,   1369,   3955,    634,      0,    745,      0,      2],
        [     9, 111444,      1,      0,      5,    194,      0,      2,      1],
        [   322,      1, 132886,    190,     94,    208,  10882,      0,      1],
        [  4241,      1,   1510,  79660,    927,      0,   3783,      0,     75],
        [   862,      0,   1290,    797,  89255,     14,    194,      0,      2],
        [   142,     78,   1179,      0,      0,  94620,     49,     45,      0],
        [  1350,      7,  23572,   7500,    268,    162,  57346,      0,      2],
        [   162,    588,     62,      0,      0,      0,      0,  89220,      0],
        [   225,     85,    298,      7,     17,      1,      2,      0, 111000]])


Monitored metric Validation/Accuracy did not improve in the last 5 records. Best score: 0.977. Signaling Trainer to stop.


Epoch end: Train, epoch number: 33
TrainEpoch/CM
 tensor([[ 80085,      0,   1369,   3955,    634,      0,    745,      0,      2],
        [     9, 111444,      1,      0,      5,    194,      0,      2,      1],
        [   322,      1, 132886,    190,     94,    208,  10882,      0,      1],
        [  4241,      1,   1510,  79660,    927,      0,   3783,      0,     75],
        [   862,      0,   1290,    797,  89255,     14,    194,      0,      2],
        [   142,     78,   1179,      0,      0,  94620,     49,     45,      0],
        [  1350,      7,  23572,   7500,    268,    162,  57346,      0,      2],
        [   162,    588,     62,      0,      0,      0,      0,  89220,      0],
        [   225,     85,    298,      7,     17,      1,      2,      0, 111000]])


In [24]:
model.eval()

MalwaresModelMLP(
  (train_metrics): MetricCollection(
    (MulticlassAccuracy): MulticlassAccuracy()
    (MulticlassPrecision): MulticlassPrecision()
    (MulticlassRecall): MulticlassRecall()
    (MulticlassF1Score): MulticlassF1Score(),
    prefix=Train/
  )
  (val_metrics): MetricCollection(
    (MulticlassAccuracy): MulticlassAccuracy()
    (MulticlassPrecision): MulticlassPrecision()
    (MulticlassRecall): MulticlassRecall()
    (MulticlassF1Score): MulticlassF1Score(),
    prefix=Validation/
  )
  (test_metrics): MetricCollection(
    (MulticlassAccuracy): MulticlassAccuracy()
    (MulticlassPrecision): MulticlassPrecision()
    (MulticlassRecall): MulticlassRecall()
    (MulticlassF1Score): MulticlassF1Score(),
    prefix=Test/
  )
  (cm): MulticlassConfusionMatrix()
  (criterion): CrossEntropyLoss()
  (l1): Linear(in_features=30, out_features=30, bias=True)
  (l2): Linear(in_features=30, out_features=30, bias=True)
  (l3): Linear(in_features=30, out_features=9, bias=True)
)

In [25]:
print(checkpoint_callback.best_model_path)   # prints path to the best model's checkpoint
print(checkpoint_callback.best_model_score) # and prints it score
best_model = MalwaresModelMLP.load_from_checkpoint(checkpoint_callback.best_model_path, input_size = vectors.shape[1]-1, output_size = len(label_encoder.classes_))

trainer.test(best_model, dataloaders=val_dataloader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


/home/<User>/repos/csg_is/training/sklearn_linear/lightning_logs/version_5/checkpoints/epoch=28-step=1247.ckpt
tensor(0.9767, device='cuda:0')


Testing: |          | 0/? [00:00<?, ?it/s]

Epoch end: Test, epoch number: 0
TestEpoch/CM
 tensor([[513,   0,   0,   5,   0,   0,   0,   0,   0],
        [  0, 646,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0, 802,   0,   0,   0,  43,   0,   0],
        [  0,   0,   0, 497,   0,   0,  23,   0,   0],
        [  0,   0,   0,   0, 523,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0, 571,   0,   0,   0],
        [  3,   0,  41,  15,   0,   0, 495,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0, 556,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0, 636]])
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      Test/Accuracy         0.9766867160797119
      Test/F1Score          0.9764164686203003
        Test/Loss           0.07035357505083084
     Test/Precision         0

[{'Test/Loss': 0.07035357505083084,
  'Test/Accuracy': 0.9766867160797119,
  'Test/Precision': 0.9762660264968872,
  'Test/Recall': 0.9766867160797119,
  'Test/F1Score': 0.9764164686203003,
  'TestEpoch/Accuracy': 0.9765256643295288,
  'TestEpoch/Precision': 0.9765797853469849,
  'TestEpoch/Recall': 0.9765256643295288,
  'TestEpoch/F1Score': 0.9765474200248718}]

In [26]:
print(checkpoint_callback.best_model_path)   # prints path to the best model's checkpoint
print(checkpoint_callback.best_model_score) # and prints it score
best_model = MalwaresModelMLP.load_from_checkpoint("/home/<User>/repos/csg_is/training/sklearn_linear/lightning_logs/version_4/checkpoints/epoch=97-step=4116.ckpt", input_size = vectors.shape[1]-1, output_size = len(label_encoder.classes_))

trainer.test(best_model, dataloaders=val_dataloader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


/home/<User>/repos/csg_is/training/sklearn_linear/lightning_logs/version_5/checkpoints/epoch=28-step=1247.ckpt
tensor(0.9767, device='cuda:0')


Testing: |          | 0/? [00:00<?, ?it/s]

Epoch end: Test, epoch number: 0
TestEpoch/CM
 tensor([[518,   0,   0,   0,   0,   0,   0,   0,   0],
        [  0, 646,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0, 811,   0,   0,   0,  34,   0,   0],
        [  0,   0,   0, 496,   0,   0,  24,   0,   0],
        [  0,   0,   0,   0, 523,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0, 571,   0,   0,   0],
        [  0,   0,   1,   8,   0,   0, 545,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0, 556,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0, 636]])
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      Test/Accuracy         0.9888604879379272
      Test/F1Score          0.9878873825073242
        Test/Loss           0.04431302472949028
     Test/Precision         0

[{'Test/Loss': 0.04431302472949028,
  'Test/Accuracy': 0.9888604879379272,
  'Test/Precision': 0.9875195622444153,
  'Test/Recall': 0.9888604879379272,
  'Test/F1Score': 0.9878873825073242,
  'TestEpoch/Accuracy': 0.9885960221290588,
  'TestEpoch/Precision': 0.98741215467453,
  'TestEpoch/Recall': 0.9885960221290588,
  'TestEpoch/F1Score': 0.9877465963363647}]